In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
# Load the dataset
url = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df = pd.read_csv(url)
df.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


In [3]:
# Data preprocessing
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

In [4]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(np.array(new_data['Close']).reshape(-1,1))

In [5]:
# Create training and testing datasets
train_len = int(len(scaled_data)*0.8)
train_data = scaled_data[0:train_len,:]
x_train, y_train = [], []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [6]:
# Reshape the data for LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [7]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [8]:
# Compile and fit the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)



1568/1568 - 42s - loss: 0.0020 - 42s/epoch - 27ms/step


In [9]:
# Create testing dataset
test_data = scaled_data[train_len - 60: , :]
x_test = []
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [10]:
# Predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

13/13 [==============================] - 1s 16ms/step


In [14]:
# Rescale test data for comparison
test_data_scaled = scaler.inverse_transform(test_data[-len(predictions):])

# Evaluate the model
rmse = np.sqrt(np.mean((predictions - test_data_scaled)**2))
print("Root Mean Square Error:", rmse)

13/13 [==============================] - 0s 9ms/step
Root Mean Square Error: 25535.659874579545
